# Accessibilité relative des résidus et évolution protéique

Il y a plusieurs étapes dans le traitement des données :

 * obtenir les données pdb par **PISCES**
 * faire tourner **DSSP** pour obtenir l'accessibilité du solvant
 * récupération des séquences homologues par **PSI-BLAST** (pas moins de 70% d'identité)
 * réalignement avec la séquence parente
 * repositionnement des chaines latérales avec **SCWRL4** -> accessibilité du solvant ? 

## PISCES

Sélection de la base de données la plus petite base de [données](http://dunbrack.fccc.edu/Guoli/culledpdb_hh/cullpdb_pc20_res1.6_R0.25_d201103_chains3760.gz)

Pour récupérer les fichiers PDB par ligne de commande simplement faire :  
`wget https://files.rcsb.org/download/XXXX.pdb`  
Le fichier pdb se créera dans le dossier sous le nom XXXX.pdb avec XXXX l'ID de la protéine.

In [30]:
def pdbfiles(pisces_file):
    """Creates list of [pdb id, chain, length]
    """
    with open(pisces_file, 'r') as file:
        pdb_list = []
        next(file)
        for line in file:
            line = line.split()
            pdb_list.append([line[0][0:4], line[0][4:], int(line[1])])
        return pdb_list

In [31]:
PDB = pdbfiles('../../cullpdb_pc20_res1.6_R0.25_d201103_chains3760')

On récupère ainsi une liste contenant pour chaque protéine une liste de son identifiant, la chaine choisie et sa taille.

## DSSP

In [4]:
import wget

pdbfiles = wget.download('https://files.rcsb.org/download/1AHO.pdb', out = '../data/1AHO.pdb')
pdbfiles = wget.download('https://files.rcsb.org/download/1BTE.pdb', out = '../data/1BTE.pdb')

### Traitement de 1AHO

In [32]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

p = PDBParser()
structure = p.get_structure("1AHO","../data/1AHO.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/1AHO.pdb")

In [33]:
def full_structure(DSSP_dict):
    """ Check si certains acides aminés n'ont pas de structure secondaire associés
    """
    for key in DSSP_dict.keys():
        if DSSP_dict[key][2] == '-':
            return False
    return True

In [34]:
print(full_structure(DSSP)) #si False -> certains AA n'ont pas de structure secondaire predites

False


In [35]:
ASA = 0
for k in DSSP.keys():
    #print(DSSP[k]) #affichage de chaque element pour tous les AA
    ASA += DSSP[k][3] #recuperation de la 'relative ASA' de chaque AA

print("Accessibilite relative du solvant pour la proteine entiere : {}".format(ASA))

Accessibilite relative du solvant pour la proteine entiere : 23.91777136977158


In [36]:
# Recuperation de la sequence fasta de la proteine
fasta = ""
for key in DSSP.keys():
    fasta += DSSP[key][1] #recuperation du symbole 1 lettre de l'AA
print(fasta)

VKDGYIVDDVNCTYFCGRNAYCNEECTKLKGESGYCQWASPYGNACYCYKLPDHVRTKGPGRCH


### Traitement de 1BTE

In [37]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

p = PDBParser()
structure = p.get_structure("1BTE","../data/1BTE.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/1BTE.pdb")

In [38]:
ASA = 0
for k in DSSP.keys():
    #print(DSSP[k]) #affichage de chaque element pour tous les AA
    ASA += DSSP[k][3] #recuperation de la 'relative ASA' de chaque AA

print("Accessibilite relative du solvant pour la proteine entiere : {}".format(ASA))

Accessibilite relative du solvant pour la proteine entiere : 34.529479532768015


In [39]:
# Recuperation de la sequence fasta de la proteine
fasta = ""
for key in DSSP.keys():
    fasta += DSSP[key][1] #recuperation du symbole 1 lettre de l'AA
print(fasta)

ETQECLFFNANWERDRTNQTGVEPCYGRRHCFATWKNISGSIEIVKQGCWLDDINCYDRTDCIEKKDSPEVYFCCCEGNMCNEKFSYFPEM


Les calculs d'accessibilité au solvant sont faits de la même façon avec les autres fichiers PDB.